In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import math 
import pandas as pd
from datetime import datetime, timedelta
import os

%matplotlib inline
%matplotlib notebook

sns.set_context('talk')

In [ ]:
data_path = '/home/bhanu/work/hrishikeshn/'
animal_ids = ['G394', 'G396', 'G404', 'G405', 'G492', 'G493', 'G506', 'G508']
colors = sns.color_palette("colorblind", 6)
colors.append(colors[0])
protocols = ['SoAn1', 'An1', 'An2', 'An3', 'Hr7', 'All3', 'Hr6']
protocol_info = {
                'SoAn1' : '250ms',
                'An1'   : '350ms',
                'An2'   : '450ms',
                'An3'   : '550ms',
                'Hr7'   : '250ms-550ms interleaved',
                'All3'  : 'extinction'
                }
color_dict = dict(zip(protocols, colors))
probe = False 

In [ ]:
# data_path = '/home/bhanu/work/soumyab/'
# animal_ids = ['G313', 'G377']
# colors = sns.color_palette("colorblind", 3)
# colors.append(colors[0])
# protocols = ['SoAn1', 'So1', 'So2']
# protocol_info = {
#                 'SoAn1' : 'light',
#                 'So1'   : 'sound',
#                 'So2'   : 'multics',
#                 }
# color_dict = dict(zip(protocols, colors))

In [ ]:
def read_behavior_data(session_file):
    try:
        data = np.load(session_file, allow_pickle=True)
    except FileNotFoundError:
        print(session_file + ' file does not exist')
        return None 
    data = data.item()
    return data

In [ ]:
def plot_eye_blink_traces(ax, data):
    for t, br in enumerate(data['blink_response']):
#         print(data['arduino_timestamp'][0])
#         time_array = np.array([(data['arduino_timestamp'][t][i] - data['arduino_timestamp'][t][0]).microseconds for i in range(len(data['arduino_timestamp'][t]))])
        ax.plot(np.arange(len(br)),br)
    return

In [ ]:
def get_learning_stats(data):
    conditioned_response = []
    try:
        for t, br in enumerate(data['blink_response']):
            try:
                trace_frames = np.where(np.array(data['trial_phase'][t])==3)[0][:-5]
                trace_start = trace_frames[0]#data['trial_phase'][t].index(3) 
                trace_end = trace_frames[-1]#data['trial_phase'][t].index(4) 
            except:
                continue
            if np.any(np.array(br[trace_start:trace_end])>2):
                conditioned_response.append(1)
            else:
                conditioned_response.append(0)

        if len(conditioned_response)>0:
            learnt_percent = sum(conditioned_response)*100/len(conditioned_response)
        else:
            learnt_percent = math.nan
        return learnt_percent
    
    except ValueError:
        print("returning nan")
        return math.nan
        

In [ ]:
def calc_cr_peak_timing(data, probe_flag):
    conditioned_response = []
    peak_times = []
        
    for t, br in enumerate(data['blink_response']):
        if probe_flag and data['probe_flag'][t] == 0:
            continue
        try:
            trace_frames = np.where(np.array(data['trial_phase'][t])==3)[0]
        except:
            continue
        if np.any(np.array(br[trace_frames])>2):
#             print(data['trial_phase'][t])
            
            try:
                cs_onset = data['arduino_timestamp'][t][data['trial_phase'][t].index(2)]
            except:
                continue
#             print(t)
            peak_t  = data['arduino_timestamp'][t][np.argmax(br[trace_frames]) + trace_frames[0]]
            peak_times.append(((peak_t-cs_onset).seconds*1e6 + (peak_t-cs_onset).microseconds)/1000)
            
            trace_start_t  = data['arduino_timestamp'][t][trace_frames[0]]
            trace_end_t  = data['arduino_timestamp'][t][trace_frames[-1]]
            
            cr_onset_t = trace_start_t + 0.6*(trace_end_t - trace_start_t)
            if (peak_t >= cr_onset_t) and (peak_t <= trace_end_t):
                conditioned_response.append(1)
            else:
                conditioned_response.append(0)
                
                
        else:
            conditioned_response.append(0)
#             peak_times.append(math.nan)

    if len(conditioned_response)>0:
        learnt_percent = sum(conditioned_response)*100/len(conditioned_response)
    else:
        learnt_percent = math.nan
    return learnt_percent, conditioned_response, peak_times

        

In [ ]:
# n_cols = 3
# if len(animal_ids)%n_cols == 0:
#     n_rows = int(len(animal_ids)/n_cols)
# else:
#     n_rows = int(len(animal_ids)/n_cols) + 1
    
# row, col = 0, 0

# f, ax = plt.subplots(n_rows, n_cols, figsize=(15,10), sharey=True, sharex=True)    
lines = []
proto_list = []

for a, animal_id in enumerate(animal_ids):
    print(animal_id)
    metadata = pd.read_csv(data_path + 'csv/' + animal_id + '.csv', delimiter=',',
                      dtype={'date':object})
    sess_codes = []
    peak_time_values = []
    performance_values = []
    upi_values = []
    for s in range(len(metadata)): 
#         print(s)

        session_file = data_path + 'behavior_data/' + animal_id + '/' + animal_id + \
                        '_' + metadata['behaviour_code'].iloc[s] + '_' + str(metadata['behaviour_session_number'].iloc[s]) + \
                        '_behavior_data.npy'
        if not(os.path.isfile(session_file)) or ('All1' in session_file) or ('All4' in session_file) or ('So2' in session_file) or('error' in session_file) or (os.path.getsize(session_file)< 4000000):
#         if not(os.path.isfile(session_file)) or ('All1' in session_file) or ('All4' in session_file) or('error' in session_file) or (os.path.getsize(session_file)< 4000000):
            continue
#         print(session_file)
        data = read_behavior_data(session_file)
        lp, cr, pt = calc_cr_peak_timing(data, probe_flag=probe)
        peak_time_values.append(pt)
        performance_values.append(lp)
        sess_codes.append(metadata['behaviour_code'].iloc[s])
        upi_values.append(metadata['upi'].iloc[s])
        
    upi_values = np.array(upi_values) 
#     peak_time_values = np.array(peak_time_values)
    performance_values = np.array(performance_values)
    sess_codes = np.array(sess_codes)
    
    f, ax = plt.subplots(2, 1, figsize=(14,9))#, sharex=True)
    b_plot = ax[0].boxplot(peak_time_values, patch_artist=True)
    ax[0].set_xticklabels(upi_values)
#     ax[0,0].set_xlabel('Session_number')
    ax[0].set_ylabel('Time of peak eyeblink (ms)')
    
    for patch, sc in zip(b_plot['boxes'], sess_codes):
        patch.set_facecolor(color_dict[sc])
        
#     ax[1].scatter(performance_values, performance_dict.values(), color=[color_dict[sc] for sc in sess_codes])
#     print(upi_values)
    for sc in set(sess_codes):#protocols:
        indices = np.where(sess_codes == sc)[0]
#         print(indices)
#         print(sc, upi_values[indices])
#         if len(indices)>0:
        line, = ax[1].plot(upi_values[indices], performance_values[indices], c=color_dict[sc], marker='o')
#     ax[1].set_xticklabels(upi_values)
    ax[1].set_xlabel('Session number')
    ax[1].set_ylabel('Performance score')
    ax[1].set_xlim(upi_values[0]-0.5, upi_values[-1] + 0.5)
    ax[1].set_ylim(0, 100)
    ax[1].set_yticks(np.arange(0,101, 25))
    
    plt.suptitle(animal_id)
    if probe:
        plt.savefig('%s_probe_behavior_summary.svg'%animal_id)
    else:
        plt.savefig('%s_all_behavior_summary.svg'%animal_id)
#         f, ax = plt.subplots()
#         plot_eye_blink_traces(ax, data)
#         learning_percentage.append(get_learning_stats(data))
        
        
        
#         behav_code = session_file.split('/')[-1].split('_')[1]
#         behav_sess_num = int(session_file.split('/')[-1].split('_')[2])
#         sess_codes.append(behav_code)
    
        
#         upi.append(metadata.loc[(metadata['behaviour_code'] == behav_code) & (metadata['behaviour_session_number'] == behav_sess_num), 'upi'].iloc[0])
        

#         date = datetime.strptime(metadata.loc[(metadata['behaviour_code'] == behav_code) & (metadata['behaviour_session_number'] == behav_sess_num), 'date'].iloc[0], "%Y%m%d")
#         print(date.date())
#         dates.append(date)
        
#     sorted_learning_percentage = np.array([x for _, x in sorted(zip(upi, learning_percentage))])
#     sorted_sess_codes = np.array([x for _, x in sorted(zip(upi, sess_codes))])
#     sorted_upi = np.array(sorted(upi))
    
#     plt.figure()
    
    
#     for sc in set(sorted_sess_codes):
#         indices = np.where(sorted_sess_codes == sc)[0]
#         line, = plt.plot(sorted_upi[indices], sorted_learning_percentage[indices], c=color_dict[sc], marker='o')
#         line.set_label(sc)
#         if (animal_id=='G405') and (sc!='Hr6'):
#             lines.append(line)
#             proto_list.append(sc)
#     plt.title(animal_id)
#     plt.xlabel('session number')
#     plt.ylabel('performance score')
#     plt.xlim(0,45)
#     plt.xticks(np.arange(0,45, 10))
#     plt.ylim(0,100)
#     plt.yticks(np.arange(0,101, 25))
#     plt.savefig('%s_performance_score.svg' %animal_id)
    

# plt.figure()
# plt.legend(handles = lines, labels=[protocol_info[sc] for sc in proto_list])
# plt.savefig('legend_performance_score.svg')
      

        
    


# Summary

In [ ]:
def plot_timings(data, session_name):
    peak_times = []
    trial_list = []
        
    for t, br in enumerate(data['blink_response']):
        try:
            trace_frames = np.where(np.array(data['trial_phase'][t])==3)[0][:-3]
        except:
            continue
        if np.any(np.array(br[trace_frames])>2):
#             print(data['trial_phase'][t])
            
            try:
                cs_onset = data['arduino_timestamp'][t][data['trial_phase'][t].index(2)]
            except:
                continue
#             print(t)
            peak_t  = data['arduino_timestamp'][t][np.argmax(br[trace_frames]) + trace_frames[0]]
            peak_times.append(((peak_t-cs_onset).seconds*1e6 + (peak_t-cs_onset).microseconds)/1000)
            trial_list.append(t)
#     print(trial_list)
            
    f, ax = plt.subplots(figsize=(10,4))#, sharex=True)
    cols = []
    for i, t1 in enumerate(trial_list):
        if int(t1/5)%2==0:
            cols.append(color_dict['SoAn1'])
            print(t1, peak_times[i])
        else:
            cols.append(color_dict['An3'])
    b_plot = ax.scatter(trial_list, peak_times, color = cols)
    ax.set_xticklabels(upi_values)
#     0,0].set_xlabel('Session_number')
    ax.set_ylabel('Time of peak eyeblink (ms)')
    ax.set_title(session_name)

        
     
    return 

            

In [ ]:
def calc_performance_interleaved(data):
    stim_1_cr = []
    stim_2_cr = []
    stim_1_peak_times = []
    stim_2_peak_times = []
    stim_1_trial_list = []
    stim_2_trial_list = []
        
    for t, br in enumerate(data['blink_response']):
        try:
            trace_frames = np.where(np.array(data['trial_phase'][t])==3)[0]
        except:
            continue
        if np.any(np.array(br[trace_frames])>2):
#             print(data['trial_phase'][t])
            
            try:
                cs_onset = data['arduino_timestamp'][t][data['trial_phase'][t].index(2)]
            except:
                continue
#             print(t)
            peak_t  = data['arduino_timestamp'][t][np.argmax(br[trace_frames]) + trace_frames[0]]
            if int(t/5)%2 == 0:
                stim_1_peak_times.append(((peak_t-cs_onset).seconds*1e6 + (peak_t-cs_onset).microseconds)/1000)
                stim_1_trial_list.append(t)
            else:
                stim_2_peak_times.append(((peak_t-cs_onset).seconds*1e6 + (peak_t-cs_onset).microseconds)/1000)
                stim_2_trial_list.append(t)
            
            
            
            trace_start_t  = data['arduino_timestamp'][t][trace_frames[0]]
            trace_end_t  = data['arduino_timestamp'][t][trace_frames[-1]]
            
            cr_onset_t = trace_start_t + 0.6*(trace_end_t - trace_start_t)
            
            if (peak_t >= cr_onset_t) and (peak_t <= trace_end_t):
                if int(t/5)%2 == 0:
                    stim_1_cr.append(1)
                else:
                    stim_2_cr.append(1)
            else:
                if int(t/5)%2 == 0:
                    stim_1_cr.append(0)
                else:
                    stim_2_cr.append(0)
                
                
        else:
            if int(t/5)%2 == 0:
                stim_1_cr.append(0)
            else:
                stim_2_cr.append(0)

    if len(stim_1_cr)>0:
        stim_1_learnt_percent = sum(stim_1_cr)*100/len(stim_1_cr)
    else:
        stim_1_learnt_percent = math.nan
        
    if len(stim_2_cr)>0:
        stim_2_learnt_percent = sum(stim_2_cr)*100/len(stim_2_cr)
    else:
        stim_2_learnt_percent = math.nan
    return stim_1_learnt_percent, stim_2_learnt_percent, stim_1_peak_times, stim_2_peak_times, stim_1_trial_list, stim_2_trial_list 

         
    

In [ ]:
# n_cols = 3
# if len(animal_ids)%n_cols == 0:
#     n_rows = int(len(animal_ids)/n_cols)
# else:
#     n_rows = int(len(animal_ids)/n_cols) + 1
    
# row, col = 0, 0

# f, ax = plt.subplots(n_rows, n_cols, figsize=(15,10), sharey=True, sharex=True)    
lines = []
proto_list = []

for a, animal_id in enumerate(animal_ids):
    print(animal_id)
    metadata = pd.read_csv(data_path + 'csv/' + animal_id + '.csv', delimiter=',',
                      dtype={'date':object})
    sess_codes = []
    stim_1_peak_time_values = []
    stim_2_peak_time_values = []
    stim_1_performance_values = []
    stim_2_performance_values = []
    upi_values = []
    stim_1_trial_num_list = []
    stim_2_trial_num_list = []
    
    for s in range(len(metadata)): 
#         print(s)

        session_file = data_path + 'behavior_data/' + animal_id + '/' + animal_id + \
                        '_' + metadata['behaviour_code'].iloc[s] + '_' + str(metadata['behaviour_session_number'].iloc[s]) + \
                        '_behavior_data.npy'
        if not(os.path.isfile(session_file)) or ('Hr7' not in session_file) or ('error' in session_file) or (os.path.getsize(session_file)< 4000000):
#         if not(os.path.isfile(session_file)) or ('All1' in session_file) or ('All4' in session_file) or('error' in session_file) or (os.path.getsize(session_file)< 4000000):
            continue
#         print(session_file)
        data = read_behavior_data(session_file)
#         plot_timings(data, session_file.split('.')[0].split('/')[-1])
        lp1, lp2, pt1, pt2, tl1, tl2 = calc_performance_interleaved(data)
        peak_time_values.append(pt)
        stim_1_performance_values.append(lp1)
        stim_2_performance_values.append(lp2)
        stim_1_peak_time_values.append(pt1)
        stim_2_peak_time_values.append(pt2)
        stim_1_trial_num_list.append(tl1)
        stim_2_trial_num_list.append(tl2)
#         sess_codes.append(metadata['behaviour_code'].iloc[s])
        upi_values.append(metadata['upi'].iloc[s])
        
    upi_values = np.array(upi_values) 
    stim_1_performance_values = np.array(stim_1_performance_values)
    stim_2_performance_values = np.array(stim_2_performance_values)
    sess_codes = np.array(sess_codes)
    print(stim_1_performance_values)
    print(stim_2_performance_values)
    f, ax = plt.subplots(2, 1, figsize=(8,9))#, sharex=True)
    b_plot1 = ax[0].boxplot(stim_1_peak_time_values, patch_artist=True)
    
    for patch in b_plot1['boxes']:
        patch.set_facecolor('b')
        patch.set_alpha(0.5)
        
    b_plot2 = ax[0].boxplot(stim_2_peak_time_values, patch_artist=True)
    for patch in b_plot2['boxes']:
        patch.set_facecolor('r')
        patch.set_alpha(0.5)
#     ax[0].set_xticklabels(upi_values)
    ax[0].set_xticklabels([])
#     ax[0,0].set_xlabel('Session_number')
    ax[0].set_ylabel('Time of peak eyeblink (ms)')
    
    
        
    ax[1].plot(upi_values, stim_1_performance_values, c='b', marker='o', label='250ms')
    ax[1].plot(upi_values, stim_2_performance_values, c='r', marker='o', label='550ms')
#     ax[1].set_xticklabels(upi_values)
    ax[1].set_xlabel('Session number')
    ax[1].set_ylabel('Performance score')
    ax[1].set_xlim(upi_values[0]-0.5, upi_values[-1] + 0.5)
    ax[1].set_ylim(0, 100)
    ax[1].set_yticks(np.arange(0,101, 25))
    plt.suptitle(animal_id)
    plt.legend()
    plt.savefig('%s_hr7_behavior_summary.svg' %animal_id)
    


In [ ]:
# n_cols = 3
# if len(animal_ids)%n_cols == 0:
#     n_rows = int(len(animal_ids)/n_cols)
# else:
#     n_rows = int(len(animal_ids)/n_cols) + 1
    
# row, col = 0, 0

# f, ax = plt.subplots(n_rows, n_cols, figsize=(15,10), sharey=True, sharex=True)    
lines = []
proto_list = []

for a, animal_id in enumerate(animal_ids):
    print(animal_id)
    session_files = sorted(glob.glob(data_path + 'behavior_data/' + animal_id + '/*.npy'))
    learning_percentage = []
    upi = []
    dates = []
    sess_codes = []
    for s, session_file in enumerate(session_files):
        if ('All1' in session_file) or ('All4' in session_file) or ('So2' in session_file) or('error' in session_file) or (os.path.getsize(session_file)< 4000000):
            continue
#         print(session_file)
#         print(s, row, col)
        data = read_behavior_data(session_file)
#         plot_eye_blink_traces(ax[row, col], data)
        learning_percentage.append(get_learning_stats(data))
#         print(learning_percentage[-1])
        
        
        
        behav_code = session_file.split('/')[-1].split('_')[1]
        behav_sess_num = int(session_file.split('/')[-1].split('_')[2])
        sess_codes.append(behav_code)
#         print(behav_code, behav_sess_num)
        metadata = pd.read_csv(data_path + 'csv/' + animal_id + '.csv', delimiter=',',
                          dtype={'date':object})
    
        
#         upi = metadata.loc[(metadata['behaviour_code'] == behav_code) & (metadata['behaviour_session_number'] == behav_sess_num), 'upi'].iloc[0]
        upi.append(metadata.loc[(metadata['behaviour_code'] == behav_code) & (metadata['behaviour_session_number'] == behav_sess_num), 'upi'].iloc[0])

#         date = datetime.strptime(metadata.loc[(metadata['behaviour_code'] == behav_code) & (metadata['behaviour_session_number'] == behav_sess_num), 'date'].iloc[0], "%Y%m%d")
#         print(date.date())
#         dates.append(date)
#         color_list.append(color_dict[behav_code])
        
    sorted_learning_percentage = np.array([x for _, x in sorted(zip(upi, learning_percentage))])
    sorted_sess_codes = np.array([x for _, x in sorted(zip(upi, sess_codes))])
    sorted_upi = np.array(sorted(upi))
    
    plt.figure()
    
    
    for sc in set(sorted_sess_codes):
        indices = np.where(sorted_sess_codes == sc)[0]
        line, = plt.plot(sorted_upi[indices], sorted_learning_percentage[indices], c=color_dict[sc], marker='o')
        line.set_label(sc)
        if (animal_id=='G405') and (sc!='Hr6'):
            lines.append(line)
            proto_list.append(sc)
    plt.title(animal_id)
    plt.xlabel('session number')
    plt.ylabel('performance score')
    plt.xlim(0,45)
    plt.xticks(np.arange(0,45, 10))
    plt.ylim(0,100)
    plt.yticks(np.arange(0,101, 25))
    plt.savefig('%s_performance_score.svg' %animal_id)
    

plt.figure()
plt.legend(handles = lines, labels=[protocol_info[sc] for sc in proto_list])
plt.savefig('legend_performance_score.svg')
      

        
    


## 